## Importing Lib  ##

In [32]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Reading Data ##

In [33]:
project_dir=r"C:\Users\Bhagyashree\OneDrive\Desktop\Flights_predictions"
Data_dir="Data"

In [34]:
def get_data(name):
    file_name=f"{name}.csv"
    file_path=os.path.join(project_dir,Data_dir,file_name)
    return pd.read_csv(file_path)
    
    
    


In [35]:
flights=get_data("flight_price")

In [36]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


# Obervation #

--data contains 1063 rows and 11 features
--date_of journey,arival_time,dep_time  should be in date/time format need to handle this
--Routr and total_stops have missing values
-- duration can be changed into mintues so it should be numeric value
-- total_stops should be numeric like 0,1,2 but its str
--220 duplicates should be removed

-Check data_types -

In [37]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [38]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [39]:
value=flights.Date_of_Journey.iloc[9]
value

'27/05/2019'

In [9]:
# Duplicates#

In [40]:
flights.duplicated().sum()

220

In [41]:
(
    flights
     .loc[flights.duplicated(keep=False)]
    .sort_values(["Airline","Date_of_Journey","Source","Destination"])
)
    

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


# Detailed Analysis #

- Airline 

In [42]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [43]:
(
    flights
    .Airline
    .str.replace("Premium economy","")
    .str.replace("Business" ,"")
    .str.title()
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [14]:
--some of entries have inconsistence value 

SyntaxError: invalid syntax (2403218520.py, line 1)

- date_of_journey 


In [44]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [45]:
pd.to_datetime(flights.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

- Source

In [46]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

- Destination 

In [47]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

- Route

- as in route there is same info in total_stops so we can drop it as its not relevant to keep that -


- Departure_time

In [48]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

# checking if dep_time has diff value other than : or digits
(
    flights
    .Dep_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
)

In [49]:
pd.to_datetime(flights.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

- Arival Time

In [50]:
(
    flights
    .Arrival_Time
    .loc[lambda ser:ser.str.contains("[^0-9:]")]
    .str.split(" ",n=1)#splittting only on first
    .str.get(1)
    .unique()
    
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

- Duration 

In [51]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour=lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
         minute=lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna('0')
            .astype(int)
         )
            
        
        )
    .sum(axis=1) # every value to minutes
    
    
)


0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [52]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand=True)
    .set_axis(["hour","minute"],axis=1)
    .assign(
        hour=lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
         minute=lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna('0')
            .astype(int)
         )
            
        
        )
    .sum(axis=1) # every value to minutes
    .rename("duration_minutes")
    .to_frame()
    
    
    
)


,duration_minutes
0,170
1,445
2,1140
3,325
4,285
...,...
10678,150
10679,155
10680,180
10681,160


In [24]:
(
    flights
    .Duration
    .loc[lambda ser:~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [53]:
(
    flights
    .Duration
    .loc[lambda ser:~ser.str.contains("h")]
    #.unique()
)

6474    5m
Name: Duration, dtype: object

- Total_stops
  

In [54]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace("stops?","",regex=True) # search for stops or " stops"
    .pipe(lambda ser:pd.to_numeric(ser))
    
)
#as it has null value so that we can change its type as null is float

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

In [75]:
flights.aditional_info

AttributeError: 'DataFrame' object has no attribute 'aditional_info'

# Cleaning operation #

In [83]:
def clean_data(df):
    return (
        df
            
            .drop(index=[6474])
            .drop_duplicates()
            
       
        .assign(
            **{
               col:df[col].str.strip()
                for col in df.select_dtypes(include='O').columns 
            })
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_:(
                df_
                .airline
                .str.replace("Premium economy","")
                .str.replace("Business" ,"")
                .str.title()
            ),
            date_of_journey=lambda df_:pd.to_datetime(df_.date_of_journey,dayfirst=True),
            dep_time=lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            arrival_time= lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_:df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_:(
            df_
            .total_stops
            .replace("non-stop","0")
            .str.replace("stops?","",regex=True) # search for stops or " stops"
            .pipe(lambda ser:pd.to_numeric(ser))
        ),
            additional_info= lambda df_:df_.additional_info.replace("No info","no info")
                
        )
        .drop(columns='route')
    

        )
            
    

    

In [82]:
def convert_to_minutes(ser):
    return(
        ser
        .str.split(" ",expand=True)
       .set_axis(["hour","minute"],axis=1)
        .assign(
        hour=lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
         minute=lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna('0')
            .astype(int)
         )
            
        
        )
    .sum(axis=1) # every value to minutes
    )       
    

In [84]:
flights_clean=clean_data(flights)

In [85]:
flights_clean

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,no info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,no info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,no info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,no info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,no info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,no info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,no info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,no info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,no info,12648


In [86]:
flights_final=flights_clean.sample(1000)

In [87]:
X=flights_final.drop(columns='price')
y=flights_final.price.copy()

In [88]:
X_,X_test,y_,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val=train_test_split(X_,y_,test_size=0.2,random_state=42)

In [89]:
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


In [90]:
def export(X,y,name):
    file_name=f"{name}.csv"
    file_path=os.path.join(project_dir,Data_dir,file_name)
    X.join(y).to_csv(file_path,index=False)
    return pd.read_csv(file_path).head()

In [91]:
export(X_train,y_train,'train')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-09,Kolkata,Banglore,15:30:00,18:05:00,155,0.0,no info,5224
1,Multiple Carriers,2019-03-21,Delhi,Cochin,09:45:00,16:10:00,385,1.0,no info,7242
2,Multiple Carriers,2019-06-27,Delhi,Cochin,09:15:00,19:00:00,585,1.0,no info,11622
3,Jet Airways,2019-06-03,Banglore,Delhi,07:10:00,10:10:00,180,0.0,In-flight meal not included,5769
4,Goair,2019-03-03,Delhi,Cochin,07:00:00,12:50:00,350,1.0,no info,7378


In [92]:
export(X_val,y_val,'val')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-06,Kolkata,Banglore,09:35:00,19:10:00,575,1.0,In-flight meal not included,13469
1,Vistara,2019-05-12,Chennai,Kolkata,17:45:00,20:05:00,140,0.0,no info,11982
2,Indigo,2019-04-06,Banglore,Delhi,16:55:00,19:55:00,180,0.0,no info,4823
3,Indigo,2019-04-01,Mumbai,Hyderabad,16:55:00,18:25:00,90,0.0,no info,3175
4,Indigo,2019-05-15,Mumbai,Hyderabad,02:35:00,04:05:00,90,0.0,no info,2754


In [93]:
export(X_test,y_test,'test')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-06,Mumbai,Hyderabad,12:00:00,13:30:00,90,0.0,no info,2754
1,Indigo,2019-04-15,Banglore,Delhi,18:25:00,21:20:00,175,0.0,no info,4823
2,Multiple Carriers,2019-06-03,Delhi,Cochin,10:20:00,01:30:00,910,1.0,no info,6795
3,Air India,2019-06-09,Chennai,Kolkata,11:40:00,13:55:00,135,0.0,no info,4667
4,Jet Airways,2019-05-21,Kolkata,Banglore,06:30:00,07:25:00,1495,2.0,In-flight meal not included,10118
